# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Мы имеем следующие данные по заёмщикам:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Заполним недостающие данные, удалим дубликаты и выясним, как те или иные характеристики влияют на своевременный возврат кредита.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

In [2]:
scoring = pd.read_csv("data.csv")
scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
scoring.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# видим, что из ряда отрицательных значений стажа выбивается группа огромных положительных значений. Подсчитаем размер этой группы.
len(scoring[scoring["days_employed"] > 0]["days_employed"])

3445

In [5]:
# видим, что из ряда отрицательных значений стажа выбивается группа огромных положительных значений. Найдем ее границы.
print(scoring[scoring["days_employed"] > 0]["days_employed"].min())
print(scoring[scoring["days_employed"] > 0]["days_employed"].max())

328728.72060451825
401755.40047533


### Вывод

Мы видим, что у нас имеется 3445 значения, которые не могут означать количество дней стажа, так как все они значительно больше 300 000. Заметим, что наименьшее из них - 328728 - составляет почти ровно 900 лет, а наибольшее - 401755 - почти точно 1100 лет. Предположим, что дело в опечатках (к примеру, работник банка вбил "0" вместо "9"), и попробуем восстановить верные даты. Появление же отрицательных значений можно объяснить тем, что из даты начала стажа вычитали дату конца стажа, а не наоборот.

Кроме того, заметим, что у нас есть 21525 - 19351 = 2174 пропуска в столбцах стажа и дохода. Можно было бы предположить, что пропуски в столбце дохода являются неслучайными (бедные и богатые люди не любят указывать свой размер дохода), однако точное соответствие пропусков в столбце дохода пропускам в столбце стажа заставляет предположить случайный характер пропусков (в каком-то отделении банка люди заполняли неверную анкету).

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
scoring['education'] = scoring['education'].str.lower()
scoring['family_status'] = scoring['family_status'].str.lower()
# для начала приведем все значения к нижнему регистру

In [7]:
# так как у нас всего 2 предпринимателя, объединим их с компаньонами
scoring.loc[scoring["income_type"] == 'предприниматель', "income_type"] = "компаньон" 
# так как у нас всего 2 безработных, 1 человек в декрете и 1 студент, объединим их с пенсионерами (все эти группы живут на пособие)
for i in ['безработный', 'студент', 'в декрете']:
    scoring.loc[scoring["income_type"] == i, "income_type"] = "пенсионер" 

In [8]:
max_exp = scoring[scoring["days_employed"] < 0]["days_employed"].min() * (-1) # находим максимальную величину "настоящего" стажа
scoring.loc[scoring['days_employed'] > (365 * 1100 - max_exp), 'days_employed'] -= (1100 * 365) # корректируем ошибку в 1100 лет стажа
scoring.loc[scoring['days_employed'] > (365 * 1000 - max_exp), 'days_employed'] -= (1000 * 365) # корректируем ошибку в 1000 лет стажа
scoring.loc[scoring['days_employed'] > (365 * 900 - max_exp), 'days_employed'] -= (900 * 365) # корректируем ошибку в 900 лет стажа
scoring['days_employed'] = scoring['days_employed'].abs() # переводим отрицательные величины стажа в положительные

In [9]:
# создадим функцию для расчёта среднего возраста по типу занятости
def mean_age_by_income_type(df, income_type):
    for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]: 
        if income_type == i:
            return df[(df["income_type"] == i) & (scoring['dob_years'] != 0)]["dob_years"].mean()

In [10]:
# найдем средний возраст по типу занятости и присвоим эти значения нулевым значениям возраста
for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]:
    scoring.loc[(scoring['dob_years'] == 0) & (scoring["income_type"] == i), 'dob_years'] = mean_age_by_income_type(scoring, i)

In [11]:
# создадим функцию для расчёта среднего стажа по типу занятости
def mean_exp_by_income_type(df, income_type):
    for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]: 
        if income_type == i:
            return df[df['income_type'] == i]['days_employed'].mean()

In [12]:
# найдем средний стаж по типу занятости и присвоим эти значения пропускам
for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]:
    scoring.loc[scoring["income_type"] == i, "days_employed"] = scoring.loc[scoring["income_type"] == i, "days_employed"].fillna(mean_exp_by_income_type(scoring, i))

In [13]:
# создадим функцию для расчёта среднего дохода по типу занятости
def mean_income_by_income_type(df, income_type):
    for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]: 
        if income_type == i:
            return df[df['income_type'] == i]['total_income'].median()

In [14]:
# найдем средний доход по типу занятости и присвоим эти значения пропускам
for i in ["пенсионер", "госслужащий", "компаньон", "сотрудник"]:
    scoring.loc[scoring["income_type"] == i, "total_income"] = scoring.loc[scoring["income_type"] == i, "total_income"].fillna(scoring.loc[scoring["income_type"] == i, 'total_income'].median())

In [15]:
scoring.loc[scoring['children'] == 20, 'children'] = 2 #корректируем аномальное количество детей
scoring.loc[scoring['children'] == -1, 'children'] = 1 #корректируем аномальное количество детей

### Вывод

Все пропуски в столбцах "стаж", "доход", нулевые значения в столбце "возраст" исправлены на средние значения по типу занятости, а аномальное количество детей - на нормальное.

### Замена типа данных

In [16]:
scoring['days_employed'] = scoring['days_employed'].astype('int')
scoring['total_income'] = scoring['total_income'].astype('int')
scoring['dob_years'] = scoring['dob_years'].astype('int')

### Вывод

Из общей информации о таблице (шаг 1) мы видим, что столбцы стажа и дохода имеют вещественный тип. Заменим его на целочисленный.
Использован метод .astype()

### Обработка дубликатов

In [17]:
print(scoring.duplicated().sum())
scoring = scoring.drop_duplicates().reset_index(drop = True)

71


### Вывод

Возможные причины появления дубликатов: случайный copy-paste, случайные опечатки в ФИО, которые привели к созданию "клонов". Так как строк-дубликатов всего 71, просто избавимся от них.

### Лемматизация

In [18]:
# from pymystem3 import Mystem
# m = Mystem()

In [19]:
# scoring['purpose'].unique() # изучим основные цели кредита и попробуем их сгруппировать при помощи лемматизации
# # заметно, что цели можно объединить в "свадьбу", "покупку жилья", "покупку авто", "образование", "коммерческую недвижимость"

In [20]:
# def lemming(row): #создадим функцию для лемматизации цели кредита
#     lemmas = m.lemmatize(row['purpose'])
#     if "свадьба" in lemmas:
#         return "свадьба"
#     if "автомобиль" in lemmas:
#         return "автомобиль"
#     if "образование" in lemmas:
#         return "образование"
#     if "коммерческий" in lemmas:
#         return "коммерческая_недвижимость"
#     if "жилье" in lemmas:
#         return "жильё"
#     if "недвижимость" in lemmas:
#         return "жильё"
# scoring['purpose_lemm'] = scoring.apply(lemming, axis=1) #выводим лемматизированные значения цели кредита в отдельный столбец

### Вывод

Разнообразные цели получения кредита сведены к нескольким стандартным.

### Категоризация данных

In [21]:
def if_child(row): #разделим всех заемщиков на бездетных (индекс 0) и имеющих детей (индекс 1)
    child_index = row['children']
    if child_index == 0:
        return 0
    return 1
scoring['child_index'] = scoring.apply(if_child, axis=1)

In [22]:
def if_ever_married(row): #разделим всех заемщиков на оформлявших свои отношения (индекс 1) и не оформлявших (индекс 0)
    family_status = row['family_status']
    if (family_status == "женат / замужем") or (family_status == "в разводе") or (family_status == "вдовец / вдова"):
        return 1
    return 0
scoring['family_status_id'] = scoring.apply(if_ever_married, axis=1)

In [23]:
import numpy as np

In [24]:
a = np.array(scoring['total_income']) #посчитаем квантили заемщиков по доходам
low20 = np.percentile(a, 20)
low40 = np.percentile(a, 40)
low60 = np.percentile(a, 60)
low80 = np.percentile(a, 80)

In [25]:
def richness(row):
    if row["total_income"] <= low20:
        return("1 квантиль")
    if row["total_income"] <= low40:
        return("2 квантиль")
    if row["total_income"] <= low60:
        return("3 квантиль")    
    if row["total_income"] <= low80:
        return("4 квантиль")
    return("5 квантиль")
scoring['quantile'] = scoring.apply(richness, axis=1)

### Вывод

Получатели кредита категоризированы на типы "с детьми"/"без детей", "регистрировавшие свои отношения"/"не бывавшие в официальном браке" и поделены на квантили по доходам.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
scoring.groupby('child_index')['debt'].mean()

child_index
0    0.075438
1    0.092082
Name: debt, dtype: float64

### Вывод

Наличие детей серьезно увеличивает вероятность несвоевременного возврата.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
scoring.groupby('family_status_id')['debt'].mean()

family_status_id
0    0.095101
1    0.074450
Name: debt, dtype: float64

### Вывод

Люди, когда-либо регистрировавшие свои отношения (женатые/замужние, разведенные, вдовые), более ответственно относятся к платежам, чем те, кто этого не делал (холостые и живущие в гражданском браке)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
#для большей наглядности продемонстрируем зависимость при помощи сводной таблицы, а не группировки 
scoring.pivot_table(index='quantile', values='debt', aggfunc='mean', margins=True) 

,debt
quantile,
1 квантиль,0.080168
2 квантиль,0.084130
3 квантиль,0.087413
4 квантиль,0.084130
5 квантиль,0.069914
All,0.081150


### Вывод

Наиболее дисциплинированно платят свои долги самые богатые (5 квантиль), и, как ни парадоксально, самые бедные - 1 квантиль (кроме госслужащих). Хуже всего - 3 квантиль.

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
#scoring.groupby('purpose_lemm')['debt'].mean()

### Вывод

Платежи по ипотеке выплачивают надежнее всего (никому не хочется потерять своё жильё).
Также хорошо выплачиваются кредиты на коммерческую недвижимость (видимо, бизнесмены хорошо рассчитывают свои финансы).
Относительно неплохо выплачиваются кредиты на свадьбу (можно предположить, что этому способствует небольшой размер кредита и помощь родителей жениха и невесты).
Хуже всего платят кредиты на образование (сложно получить хороший доход сразу после выпуска) и автомобиль (это пока еще всё-таки роскошь для населения).

### Шаг 4. Общий вывод

В ходе работы были проанализированы недостатки входящих данных о заёмщиках (пропуски, дубликаты, артефакты), причины их появления (опечатки и неполный сбор информации). Отсутствующие данные по возрасту, стажу и доходу были заполнены при помощи средних показателей по группе занятости заемщика (пенсионер, компаньон, госслужащий, сотрудник). При помощи лемматизации данных были приведены к стандартному виду цели кредита.

После предобработки данных заемщики были разделены на категории по наличию детей, семейному положению, цели кредита и разбиты на квантили по уровню дохода. 

Полученный результат показал, что появление детей резко увеличивает вероятность появления задолженности по кредиту (с 7,5 % до 9,2 %). В то же время те, кто когда-либо официально вступал в брак, более надежные заемщики, чем те, кто предпочитает холостую жизнь или свободные отношения (7,4 % должников против 9,5 %).

Что касается целей кредита, наиболее стабильно выплачиваются ипотечные кредиты (7,1% должников), хуже всего - кредиты на образование и автомобиль (9,2% и 9,4% соответственно). 

Если говорить о зависимости задолженности от уровня дохода, то наиболее надежными заёмщиками являются 20% наиболее обеспеченных людей (всего лишь 6,9 % из них вовремя не выплачивают займ). Наибольший процент задолженности - у среднего в математическом смысле класса, третьего квантиля (8,7 %), что демонстрирует опережающий рост потребностей при недостаточном уровне дохода у этой группы.